In [13]:
%load_ext autoreload
%autoreload 2

In [14]:
from datatorch.agent.flows import Flow
import os
import logging
import asyncio

logging.basicConfig(
    format="%(levelname)-8s %(message)s",
    level=logging.DEBUG,
)

In [15]:
flow = Flow.from_yaml('flow.yaml')
task = asyncio.create_task(flow.run(0))

DEBUG    Action found locally (datatorch/example-command@1).
INFO     Running action datatorch/example-command@1
DEBUG    Finished running 'Command Example' v1
DEBUG    Action found locally (datatorch/example-commands@1).
INFO     Running action datatorch/example-commands@1


b'You are using python v3.8.2 with CPython on .\n'


DEBUG    Finished running 'Multiple Command Example' v1
DEBUG    Action found locally (datatorch/example-shell@1).
INFO     Running action datatorch/example-shell@1
DEBUG    Finished running 'Shell Script Example' v1
INFO     Successfully completed job.


b'line 1\n'
b'line 2\n'
b'line 3\n'
b'line 4\n'
b'line 5\n'
b'line 6\n'


In [4]:
import asyncio
import concurrent.futures

def blocking_io():
    # File operations (such as logging) can block the
    # event loop: run them in a thread pool.
    with open('/dev/urandom', 'rb') as f:
        return f.read(100)

def cpu_bound():
    # CPU-bound operations will block the event loop:
    # in general it is preferable to run them in a
    # process pool.
    return sum(i * i for i in range(10 ** 7))

async def main():
    loop = asyncio.get_running_loop()

    ## Options:

    # 1. Run in the default loop's executor:
    result = await loop.run_in_executor(
        None, blocking_io)
    print('default thread pool', result)

    # 2. Run in a custom thread pool:
    with concurrent.futures.ThreadPoolExecutor() as pool:
        result = await loop.run_in_executor(
            pool, blocking_io)
        print('custom thread pool', result)

    # 3. Run in a custom process pool:
    with concurrent.futures.ProcessPoolExecutor() as pool:
        result = await loop.run_in_executor(
            pool, cpu_bound)
        print('custom process pool', result)

In [11]:
import time
round(time.time()*1000)

1594154237952

In [7]:
import aiodocker

In [30]:
async def docker_testing():
    docker = aiodocker.Docker()
    container = await docker.containers.run({ 'Image': 'hello-world2' })
    await container.start()
#     logs = await container.log(stdout=True, follow=True)
    async for log in container.log(stdout=True, follow=True):
        print(log.strip().strip('\n'))
    await container.delete(force=True)
    await docker.close()

task = asyncio.create_task(docker_testing())

ERROR    Task exception was never retrieved
future: <Task finished name='Task-125' coro=<docker_testing() done, defined at <ipython-input-29-97b68cf03d4d>:1> exception=DockerError(404, "pull access denied for hello-world2, repository does not exist or may require 'docker login': denied: requested access to the resource is denied")>
Traceback (most recent call last):
  File "/home/justin/.local/lib/python3.8/site-packages/aiodocker/containers.py", line 67, in run
    container = await self.create(config, name=name)
  File "/home/justin/.local/lib/python3.8/site-packages/aiodocker/containers.py", line 54, in create
    data = await self.docker._query_json(
  File "/home/justin/.local/lib/python3.8/site-packages/aiodocker/docker.py", line 293, in _query_json
    async with self._query(
  File "/home/justin/.local/lib/python3.8/site-packages/aiodocker/utils.py", line 309, in __aenter__
    resp = await self._coro
  File "/home/justin/.local/lib/python3.8/site-packages/aiodocker/docker.py",